# 前期後半の課題

## 第8回

次のコードを実行しデータセットを取得しなさい。

In [1]:
import seaborn as sns

penguins = sns.load_dataset("penguins")
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


ここでは、全結合のニューラルネットで分類モデルを定義する。`penguins`のデータセットにおいて、ターゲットを3クラスの`speicies`、説明変数を`species`以外の変数にする。

### 問1

3値分類のニューラルネットで訓練できるよう、`penguins`のデータセットに対して前処理を行いなさい。また、PyTorchのニューラルネットで学習できるように、データセットの型を変換して、`train`と`val`と`test`に3分割しなさい。

【補足】  
第9回の講義ではここで作ったデータセットを使って、モデルを訓練して評価します。良いモデルとなるように工夫して前処理しましょう。

In [2]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [6]:
# 各カラムにおける欠損値の数を調べる
missing_values = penguins.isnull()
missing_values.sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [7]:
# 欠損値が存在するフィールドの数を調べる
missing_values.sum().sum()

19

In [9]:
# 欠損値が存在するフィールドを削除
penguins.dropna(inplace=True)
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [11]:
# 欠損値が存在するフィールドの数を調べる
penguins.isnull().sum().sum()

0

In [12]:
# speciesのカラムの値を0をAdelie、1をChinstrap、2をGentooに対応させる
species_mapping = {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}

# speciesカラムの値を指定した数値に変換
penguins['species'] = penguins['species'].map(species_mapping)
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,0,Torgersen,39.5,17.4,186.0,3800.0,Female
2,0,Torgersen,40.3,18.0,195.0,3250.0,Female
4,0,Torgersen,36.7,19.3,193.0,3450.0,Female
5,0,Torgersen,39.3,20.6,190.0,3650.0,Male


In [13]:
# islandとsexのカラムをワンホットエンコードする
import pandas as pd

penguins = pd.get_dummies(penguins, columns=['island', 'sex'])
penguins.head()


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
0,0,39.1,18.7,181.0,3750.0,False,False,True,False,True
1,0,39.5,17.4,186.0,3800.0,False,False,True,True,False
2,0,40.3,18.0,195.0,3250.0,False,False,True,True,False
4,0,36.7,19.3,193.0,3450.0,False,False,True,True,False
5,0,39.3,20.6,190.0,3650.0,False,False,True,False,True


### 問2

`penguins`のデータセットに対して、3値分類のニューラルネットワークモデルをPytorchで定義しなさい。ただし、3層以上の全結合ニューラルネットで、活性化関数はすべてReLU関数にすること。ただし、学習は行わなくて良い。

### 問3

作ったニューラルネットワークモデルに、`test`のいずれかのデータを入力して、正しく動作することを確認しなさい。

## 第9回

テーブルデータに対して、全結合層のニューラルネットワークで分類モデルを作成する。


1. テーブルデータには、Pythonライブラリの`seaborn.load_dataset("penguins")`を用いる。説明変数を `['island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex']`、目的変数を`species`とする。データの前処理を行い、全結合層のニューラルネットワークモデルで学習を行うためのデータセットを作成しなさい。

In [1]:
# データセットを準備
import seaborn
df = seaborn.load_dataset("penguins")

In [7]:
# 欠損行を調べる
df[df.isnull().any(axis=1)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN
11,Adelie,Torgersen,37.8,17.3,180.0,3700.0,NaN
47,Adelie,Dream,37.5,18.9,179.0,2975.0,NaN
246,Gentoo,Biscoe,44.5,14.3,216.0,4100.0,NaN
286,Gentoo,Biscoe,46.2,14.4,214.0,4650.0,NaN
324,Gentoo,Biscoe,47.3,13.8,216.0,4725.0,NaN
336,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,NaN


In [8]:
# 欠損行を削除
df.dropna(inplace=True)
df[df.isnull().any(axis=1)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex


In [12]:
# ラベルエンコード
from sklearn.preprocessing import LabelEncoder

for col in ["species", "island", "sex"]:
    df[col] = LabelEncoder().fit_transform(df[col])
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,1
1,0,2,39.5,17.4,186.0,3800.0,0
2,0,2,40.3,18.0,195.0,3250.0,0
4,0,2,36.7,19.3,193.0,3450.0,0
5,0,2,39.3,20.6,190.0,3650.0,1
...,...,...,...,...,...,...,...
338,2,0,47.2,13.7,214.0,4925.0,0
340,2,0,46.8,14.3,215.0,4850.0,0
341,2,0,50.4,15.7,222.0,5750.0,1
342,2,0,45.2,14.8,212.0,5200.0,0


In [13]:
df.describe()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,0.918919,0.651652,43.992793,17.164865,200.966967,4207.057057,0.504505
std,0.889718,0.714715,5.468668,1.969235,14.015765,805.215802,0.500732
min,0.000000,0.000000,32.100000,13.100000,172.000000,2700.000000,0.000000
25%,0.000000,0.000000,39.500000,15.600000,190.000000,3550.000000,0.000000
50%,1.000000,1.000000,44.500000,17.300000,197.000000,4050.000000,1.000000
75%,2.000000,1.000000,48.600000,18.700000,213.000000,4775.000000,1.000000
max,2.000000,2.000000,59.600000,21.500000,231.000000,6300.000000,1.000000


In [14]:
# 正規化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']] = scaler.fit_transform(df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']])
df.describe()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
count,333.000000,333.000000,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,333.000000
mean,0.918919,0.651652,3.840772e-16,6.401286e-16,2.133762e-16,-1.707010e-16,0.504505
std,0.889718,0.714715,1.001505e+00,1.001505e+00,1.001505e+00,1.001505e+00,0.500732
min,0.000000,0.000000,-2.177987e+00,-2.067291e+00,-2.069852e+00,-1.874435e+00,0.000000
25%,0.000000,0.000000,-8.227879e-01,-7.958519e-01,-7.836512e-01,-8.172292e-01,0.000000
50%,1.000000,1.000000,9.288742e-02,6.872642e-02,-2.834620e-01,-1.953432e-01,1.000000
75%,2.000000,1.000000,8.437412e-01,7.807321e-01,8.598276e-01,7.063915e-01,1.000000
max,2.000000,2.000000,2.858227e+00,2.204743e+00,2.146028e+00,2.603144e+00,1.000000


2. 問1のデータセットに対して、全結合層のニューラルネットワークモデルを定義しなさい。そして、訓練していない状態でテストデータを入力し、正答率を算出しなさい。

In [117]:
# 訓練データ, バリデーション, テストデータに分割

import torch
from torch.utils.data import TensorDataset, DataLoader

ds_x = torch.Tensor(df.drop(columns="species").to_numpy())
ds_y = torch.Tensor(df["species"].to_numpy())
ds = TensorDataset(ds_x, ds_y)

ds_train, ds_val, ds_test = torch.utils.data.random_split(dataset=ds, lengths=[233, 50, 50], generator=torch.Generator().manual_seed(42))

batch_size = 32
ds_train = DataLoader(ds_train, batch_size=batch_size)
ds_val = DataLoader(ds_val, batch_size=batch_size)
ds_test = DataLoader(ds_test, batch_size=batch_size)

In [118]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [119]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 6)
        self.fc2 = nn.Linear(6, 3)

    def forward(self, x):
        x = nn.functional.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=6, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=3, bias=True)
)


In [120]:
correct = 0                                                     # 正しい予測の数
total = 0                                                       # テストデータの総数
model.eval()
with torch.no_grad():
    for x, y in ds_test:                                        # テストデータセット（ds_test）からデータ（x）とラベル（y）をバッチごとに取り出す
        outputs = model(x.to(device))                           # 入力データをモデルに渡して予測を取得
        _, predicted = torch.max(outputs, 1)                    # 最大値のインデックスを取得し、予測クラスとする
        print(predicted)
        total += y.size(0)                                      # テストデータの総数を更新
        correct += (predicted == y.to(device)).sum().item()     # 正しい予測の数を更新

accuracy = 100 * correct / total                                # 正答率を計算（正しい予測の割合をパーセンテージで表す）
print(f'正答率: {accuracy:.2f}%')                               # 正答率を表示


tensor([0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2,
        0, 0, 0, 0, 0, 0, 2, 0])
tensor([2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0])
正答率: 46.00%


3. 問2のニューラルネットワークモデルを問1のデータセットで訓練しなさい。訓練後、問2と同様にテストデータを入力し、正答率を算出しなさい。ただし、正答率があがるようにunit数を変更するなど、モデルのアーキテクチャを工夫し、過学習を防ぐために学習中の損失関数をプロットしなさい。

In [121]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [122]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y.long())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def val(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(correct, test_loss)

In [123]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}")
    train(ds_train, model, loss_fn, optimizer)
    val(ds_val, model, loss_fn)

Epoch 1
0.54 1.0610259175300598
Epoch 2
0.62 1.0211183428764343
Epoch 3
0.66 0.9837880730628967
Epoch 4
0.7 0.9487901031970978
Epoch 5
0.74 0.9159345328807831
Epoch 6
0.74 0.8850697875022888
Epoch 7
0.78 0.8560701012611389
Epoch 8
0.78 0.8288264572620392
Epoch 9
0.78 0.8032396733760834
Epoch 10
0.78 0.779216855764389
Epoch 11
0.78 0.7566682696342468
Epoch 12
0.78 0.7355063855648041
Epoch 13
0.78 0.7156457006931305
Epoch 14
0.78 0.6970023810863495
Epoch 15
0.78 0.6794944703578949
Epoch 16
0.78 0.6630429625511169
Epoch 17
0.8 0.6475716531276703
Epoch 18
0.8 0.6330077648162842
Epoch 19
0.8 0.6192823350429535
Epoch 20
0.8 0.6063300371170044
Epoch 21
0.8 0.5940898656845093
Epoch 22
0.8 0.5825047492980957
Epoch 23
0.8 0.5715213119983673
Epoch 24
0.8 0.561090499162674
Epoch 25
0.8 0.5511666536331177
Epoch 26
0.8 0.5417078733444214
Epoch 27
0.8 0.5326755344867706
Epoch 28
0.8 0.524034172296524
Epoch 29
0.8 0.5157512873411179
Epoch 30
0.8 0.5077971518039703
Epoch 31
0.8 0.5001445561647415
Epoch

In [124]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for x, y in ds_test:
        outputs = model(x.to(device))
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y.to(device)).sum().item()
        

accuracy = 100 * correct / total
print(f'正答率: {accuracy:.2f}%')

正答率: 96.00%


## 第11回

1. データセット`Fashion-MNIST`に対して、畳み込みニューラルネットワークの画像分類モデルを作成しなさい。
    - Fashion-MNIST: https://github.com/zalandoresearch/fashion-mnist
    - Fashion-MNIST を全結合ニューラルネットワークで分類したモデル： https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

- 全結合層: 線形変換 → ReLU
- 畳み込み層: 畳み込み → ReLU → Pooling

2. 分類してみたい画像データの候補をあげなさい。

## 第12回

1. 強化学習のライブラリ`https://github.com/Farama-Foundation/Gymnasium`をインストールして、Pythonファイルで`import`できることを確認しなさい。次回の講義で使います。
2. 強化学習を行ってみたいタスクをあげなさい。`https://gymnasium.farama.org`の`Environments`を参考にしてもよい。
3. 全結合層ニューラルネットについて、下記の例で確認しなさい。
    - https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
4. 畳み込みニューラルネットについて、下記の例で確認しなさい。
    - https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

※ 前期の最終試験は、全結合層ニューラルネットor畳み込みニューラルネットの実装を出題します。

In [1]:
# 1. 強化学習のライブラリのインストール・importできることの確認
import gymnasium as gym
env = gym.make("CartPole-v1")

observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

2. 強化学習を行ってみたいタスクをあげなさい。
- ブラックジャック

## 第13回

### 問1

全結合層で、入力を $6$ 次元ベクトル、出力を $12$ 次元ベクトルとなるように実装し、乱数で実際に動かしなさい。実装には`torch.nn.Linear`を使いなさい。

In [9]:
import torch
import torch.nn as nn
# モデルをインスタンス (in_features=6, out_features=12)
m = nn.Linear(6, 12)
# 6次元ベクトルの乱数
input = torch.randn(6)

output = m(input)
print(output.size())
print(output)

torch.Size([12])
tensor([ 0.2377,  0.2730,  0.0446,  0.1499, -0.2096,  0.1341,  0.1845,  0.2168,
        -0.3328, -0.0050, -0.4401,  0.4708], grad_fn=<ViewBackward0>)


### 問2

畳み込み層で、入力を " $3$ チャンネル, $28 \times 28$ の $2$ 次元データ", カーネルサイズを $(4, 4)$ , 出力チャンネルを $1$ チャンネルとなるように実装し、乱数で実際に動かしなさい。実装には`torch.nn.Conv2d`を使いなさい。

In [20]:
m = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=(4, 4))
input = torch.randn(3, 28, 28)
output = m(input)
print(input.size())

torch.Size([3, 28, 28])


In [18]:
print(output.size())
print(output)

torch.Size([1, 25, 25])
tensor([[[ 3.4915e-02, -9.4747e-01,  1.3592e-01, -8.6957e-01, -2.6183e-02,
           4.6550e-01, -4.6418e-01, -1.9362e-01,  8.6146e-01, -9.7878e-01,
           1.8047e-03,  1.3663e+00,  6.3965e-02,  1.9485e-01, -4.6085e-01,
           1.2662e-01, -1.2600e-01, -3.9953e-01,  4.4695e-01,  3.5632e-01,
           8.5603e-01,  2.7307e-02, -2.8006e-01, -4.1733e-01,  6.3724e-02],
         [ 7.0127e-01, -6.2151e-01,  4.3520e-01, -5.0372e-01,  4.6047e-01,
           3.9347e-01, -4.6726e-01, -1.6885e-01,  5.4482e-01,  2.3183e-01,
          -8.1326e-01,  4.3162e-02, -3.5952e-02,  8.5445e-01, -2.1357e-01,
           2.1841e-01, -2.5896e-01, -1.7895e-01, -1.2850e+00, -4.8548e-01,
           5.0518e-02, -1.7835e-01, -2.3017e-01,  4.4406e-01, -1.0214e-01],
         [ 1.5983e-02, -1.1564e+00, -5.7285e-01, -4.4076e-01,  9.1952e-01,
           8.6458e-01,  1.3436e-01,  1.7117e-01, -3.4787e-01, -1.2508e-01,
          -9.3661e-01, -8.7304e-01, -2.2475e-01,  1.0609e+00, -4.6408e-01,